# Training ResNet and SparseResNet on clean images

In [1]:
import sys, os
d = os.path.dirname(os.getcwd())
sys.path.insert(0, d)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch.utils.data import DataLoader

from utils.train_utils import train_model
from models.resnet import ResNet, SparseResNet
from utils.attacks import pgd

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


# Training on clean images

In [4]:
tr_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4, padding_mode='reflect'),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])
vl_transform = transforms.Compose([
    transforms.ToTensor()
])

ds = CIFAR10('../data', train=True, download=True, transform=tr_transform)
ds_test = CIFAR10('../data', train=False, download=True, transform=vl_transform)

batch_size = 200
train_dl = DataLoader(ds, batch_size, shuffle=True)
valid_dl = DataLoader(ds_test, batch_size, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


### ResNet

In [5]:
model = ResNet().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', min_lr=1e-5, patience=2, verbose=True)

train_model(model, train_dl, valid_dl, optimizer, loss_fn, scheduler, epochs=30, device=device)
torch.save(model.state_dict(), "../saved/resnet.pt")

Epoch: 1 Validation Loss: 1.4766 accuracy: 0.4274, time: 0:00:36
Epoch: 2 Validation Loss: 1.1345 accuracy: 0.5900, time: 0:00:36
Epoch: 3 Validation Loss: 0.8565 accuracy: 0.6918, time: 0:00:37
Epoch: 4 Validation Loss: 0.7652 accuracy: 0.7332, time: 0:00:35
Epoch: 5 Validation Loss: 0.6539 accuracy: 0.7715, time: 0:00:35
Epoch: 6 Validation Loss: 0.5590 accuracy: 0.8077, time: 0:00:34
Epoch: 7 Validation Loss: 0.5368 accuracy: 0.8133, time: 0:00:36
Epoch: 8 Validation Loss: 0.5030 accuracy: 0.8248, time: 0:00:35
Epoch: 9 Validation Loss: 0.4496 accuracy: 0.8486, time: 0:00:35
Epoch: 10 Validation Loss: 0.4263 accuracy: 0.8566, time: 0:00:33
Epoch: 11 Validation Loss: 0.4169 accuracy: 0.8651, time: 0:00:34
Epoch: 12 Validation Loss: 0.4055 accuracy: 0.8677, time: 0:00:33
Epoch: 13 Validation Loss: 0.3833 accuracy: 0.8702, time: 0:00:31
Epoch: 14 Validation Loss: 0.3868 accuracy: 0.8749, time: 0:00:33
epochs_no_improve: 1/4
Epoch: 15 Validation Loss: 0.3866 accuracy: 0.8722, time: 0:00

### SparseResNet

In [6]:
model = SparseResNet().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', min_lr=1e-5, patience=2, verbose=True)

train_model(model, train_dl, valid_dl, optimizer, loss_fn, scheduler, epochs=30, sparse=True, device=device)
torch.save(model.state_dict(), "../saved/sparse_resnet.pt")

Epoch: 1 Validation Loss: 1.4576 accuracy: 0.4182, time: 0:01:00
Epoch: 2 Validation Loss: 1.1062 accuracy: 0.5980, time: 0:00:59
Epoch: 3 Validation Loss: 0.9284 accuracy: 0.6793, time: 0:00:59
Epoch: 4 Validation Loss: 0.7231 accuracy: 0.7517, time: 0:00:59
Epoch: 5 Validation Loss: 0.6502 accuracy: 0.7781, time: 0:01:00
Epoch: 6 Validation Loss: 0.5646 accuracy: 0.8058, time: 0:00:59
Epoch: 7 Validation Loss: 0.5353 accuracy: 0.8172, time: 0:01:00
Epoch: 8 Validation Loss: 0.4720 accuracy: 0.8390, time: 0:00:59
Epoch: 9 Validation Loss: 0.4699 accuracy: 0.8412, time: 0:01:00
Epoch: 10 Validation Loss: 0.4289 accuracy: 0.8564, time: 0:00:58
Epoch: 11 Validation Loss: 0.4340 accuracy: 0.8565, time: 0:00:58
epochs_no_improve: 1/4
Epoch: 12 Validation Loss: 0.4114 accuracy: 0.8620, time: 0:00:59
Epoch: 13 Validation Loss: 0.4157 accuracy: 0.8609, time: 0:00:58
epochs_no_improve: 1/4
Epoch: 14 Validation Loss: 0.3994 accuracy: 0.8658, time: 0:00:59
Epoch: 15 Validation Loss: 0.4045 accur